In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import datetime

plt.rcParams["figure.figsize"] = (20, 5)

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [21]:
timestamp = "20200804-143723"

In [22]:
new_model = keras.models.load_model("version/{0}/model.h5".format(timestamp))
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 16, 128)           66560     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 8)                 1032      
_________________________________________________________________
activation (Activation)      (None, 8)                 0         
Total params: 199,176
Trainable params: 199,176
Non-trainable params: 0
_________________________________________________________________


In [23]:
test_set = pd.read_csv("data/SEG_test_dataset_4.csv")
test_set.head()

,scaled,delta,t,t+1
0,0.500051,878848,93730561040,93731439888
1,0.499278,-11182192,93730563456,93719381264
2,0.499281,-11128760,93730492360,93719363600
3,0.503443,53784064,93730537488,93784321552
4,0.503386,52884080,93731439888,93784323968


In [24]:
sample = test_set["scaled"].values[:16].reshape(1, -1, 1)
sample

array([[[0.50005119],
        [0.49927789],
        [0.49928132],
        [0.50344324],
        [0.50338553],
        [0.50076703],
        [0.50225755],
        [0.49599084],
        [0.49598955],
        [0.4999956 ],
        [0.49797315],
        [0.50007521],
        [0.50007468],
        [0.49944095],
        [0.50315846],
        [0.49997368]]])

In [25]:
result = new_model.predict(sample)
result

array([[0.4961526 , 0.4967316 , 0.493147  , 0.49728626, 0.50014895,
        0.4993288 , 0.50074106, 0.5005227 ]], dtype=float32)

In [27]:
import joblib

scaler = joblib.load("data/scaler.pkl")
scaled_result = scaler.inverse_transform(result).astype(np.int64)
scaled_result

array([[ -59926780,  -50896208, -106805048,  -42245400,    2403582,
         -10388331,   11638674,    8232455]], dtype=int64)

In [30]:
test_set[16:24]

,scaled,delta,t,t+1
16,0.499996,11776,93723099152,93723110928
17,0.500015,315576,93722797768,93723113344
18,0.496830,-49360128,93772465168,93723105040
19,0.499995,-3144,93722795024,93722791880
20,0.507981,124559856,93723110928,93847670784
21,0.499973,-341872,93723113344,93722771472
22,0.499974,-321792,93723105040,93722783248
23,0.499994,-6216,93722791880,93722785664


In [15]:
train_set = pd.read_csv("data/SEG_train_dataset_4.csv")
train_set.head()

,scaled,delta,t,t+1
0,0.398202,-1587653544,95444983808,93857330264
1,0.895775,6172925864,87674650304,93847576168
2,0.103254,-6187909720,93857313352,87669403632
3,0.103253,-6187918552,93857325056,87669406504
4,0.102577,-6198466584,93857330264,87658863680


In [20]:
categories = np.sort(train_set["delta"].value_counts().index)
categories

array([-7798345664, -7798172656, -7798052752, ...,  7798468256,
        7798506272,  7798506464], dtype=int64)

In [29]:
closest_point = categories[np.argsort(np.abs(scaled_result[0][0]-categories))][:10]
closest_point

array([-59928368, -59924968, -59933976, -59913552, -59897856, -59878084,
       -59875048, -59858944, -59851512, -60006528], dtype=int64)

In [32]:
scaled_result[0][0], test_set["delta"][16]

(-59926780, 11776)